![@mikegchambers](../../images/header.png)

# Simple Neural Network

In this notebook, we explore a simple Neural Network, also known as a Perceptron.   We use TensorFlow/Keras to build the network and scikit-learn to prep some data.

![Network](network.png)

UPDATE: Select the `conda_tensorflow2_p310` kernel when prompted. 

In [ ]:
# UPADTE: We turn off GPU support, although this does not seem to suppress all warnings! 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR) # <- Update - Reomved this line as it's no longer compatible with the version of TF used.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import random
import matplotlib.pyplot as plt

# The Data

Again, to be in complete control of the data within this example, we create it ourselves.

In this cell we create a template for the data, we will then pultiply this template by random values to get an example set of MCSD.

In [ ]:
template = np.array([[1.0,1.0,
                      0.1,0.1],
                     
                     [0.1,1.0,
                      0.1,1.0],
                     
                     [0.1,0.1,
                      1.0,1.0],
                     
                     [1.0,0.1,
                      1.0,0.1]])

Using the templates to generate samples:

In [ ]:
X = []
y = []

for _ in range(2000):
    for i in range(len(template)):
        r = np.array([random.uniform(0.5, 1), random.uniform(0.5, 1), random.uniform(0.5, 1), random.uniform(0.5, 1)])
        X.append(template[i] * r)
        y.append(i)
        
X = np.array(X)
y = np.array(y)

Now, let's visualize what we have:

In [ ]:
plt.figure(figsize=(15,5))
for i in range(16*4):
    plt.subplot(4,16,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X[i].reshape(2,2), cmap='Greys')
    plt.xlabel(y[i])
plt.show()

For the best outcome we need to shuffle the data so that the network dosn't get uses to the order in which samples come its way.  Also we could do with a small amount of data to be kept back for testing.  

We can do both actions in one step with scikit-learn:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

Now, let's visualize what we have:

In [ ]:
plt.figure(figsize=(15,5))
for i in range(16*4):
    plt.subplot(4,16,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_train[i].reshape(2,2), cmap='Greys')
    plt.xlabel(y_train[i])
plt.show()

# The Model

We are using a library within TensorFlow called Keras.  This does much of the heavy lifting involved in creating the network, leaving us to concentrate on the parts that matter to us.  

First, in this code cell, we clear the TensorFlow session.  This is only necessary if we come to rerun this cell and we want to be sure that we are starting again.

Then we use Keras Sequential to create a network with the size we want.  The will be (unless you change it) 4 input neurons, and 4 output neurons.

Finally, we call `model.complie` passing in the values we want for our `optimizer` which is the process we want to use in backpropagation, `loss` which is the type of loss or cost function we want to use, and lastly we set the metrics we want to record as the model trains.

In [ ]:
# Clear the session (incase we run multiple times)
tf.keras.backend.clear_session()

# Create a TF sequential model with Keras
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(4,)), # <- UPDATE: Minor change for version compatability. 
  tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Just like many times before, we call `model.fit`.  After passing in the data, we say how many epochs we want to run.

We also assign the output of the operation to a variable `e` so that we can get access to the metrics later on.

In [ ]:
e = model.fit(X_train, y_train, epochs=10)

Well that's more output than we are used to in a training model.

## How well did it train?
Remember when we called `fit` was assigned the output of the operation to the variable `e`.  Let's graph what we have in recorded in `e`, and get an idea of how the training went.

In [ ]:
plt.plot(e.history['loss'], c='red')
plt.plot(e.history['accuracy'], c='green') # <- Note minor change here from `acc` to `accuracy`.
plt.show()

# Test the Model

In [ ]:
test = 0

# UPADTE: 
# Ensure that the input is 2-dimensional
# `X_test[test]` is a single instance and should be reshaped to have 2 dimensions
p = model.predict([X_test[test].reshape(1, -1)])
pred = p.argmax()

plt.imshow(X_test[test].reshape(2,2), cmap='Greys')
plt.xticks([])
plt.yticks([])
plt.xlabel("{}:({})".format(pred, y_test[test]))

Oh... one more thing...

# Look inside the Model

If you're curious how I got the values out of the model to be able to draw the animations in the previous lessons, it starts here:

In [ ]:
model.get_weights()


And if you're even more curious, the code used to make the animations can be found here: https://github.com/learn-mikegchambers-com/ModelAnimation